# Twitter Data Mining

In this project, we'll go over the following topics:
- Collecting data from Twitter
- Text pre-processing using NLTK
- Analysing term frequencies
- Data Visualization
- Sentiment Analysis

This project was made by going through this (great) blog [post](https://marcobonzanini.com/2015/03/02/mining-twitter-data-with-python-part-1/).  Code extracts are mainly taken from there. I've also added pieces of code at various places following my interest. 

## Collecting data

In order to authorise our app to access Twitter on our behalf, we need to use the OAuth interface:

In [1]:
import tweepy
from tweepy import OAuthHandler

import os
consumer_key = os.environ['CONSUMER_KEY']
consumer_secret = os.environ['CONSUMER_SECRET']
access_token = os.environ['ACCESS_TOKEN']
access_secret = os.environ['ACCESS_SECRET']
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
 
api = tweepy.API(auth)

Display of the 10 tweets of the home timeline :

In [2]:
for status in tweepy.Cursor(api.home_timeline).items(5):
    # Process a single status
    print(status.text)

RT @sebastien_abis: Très heureux d'être l'invité du mois des entretiens #Euromed #IHEDN, le 12 à #Marseille et le 13 à #Paris, pour parler…
Just got off a call to thank folks who are working hard to help more Americans across the country sign up for healt… https://t.co/A9QvCySWJ8
Arabie saoudite : les paris risqués de Mohammed ben Salmane 
Le point de vue Francis Perrin, directeur de recherche… https://t.co/RoiSUyEsQf
RT @IRIS_SUP_: Félicitation aux nouveaux diplômés IRIS Sup' 2017! 👏
Près de 500 personnes étaient présentes, diplômés, parents, amis, tous…
RT @carole_gomez: Conférence ce soir sur lutte contre la manipulation des compétitions en Fr. à @InstitutIRIS avec T.Pujol, C.Kalb @ffhandb…


Display of the JSON response :

In [3]:
import json 
    
for status in tweepy.Cursor(api.home_timeline).items(1):
    # Process a single status
    print(json.dumps(status._json)[:1000])

{"created_at": "Mon Dec 11 21:13:03 +0000 2017", "id": 940328598958936065, "id_str": "940328598958936065", "text": "RT @sebastien_abis: Tr\u00e8s heureux d'\u00eatre l'invit\u00e9 du mois des entretiens #Euromed #IHEDN, le 12 \u00e0 #Marseille et le 13 \u00e0 #Paris, pour parler\u2026", "truncated": false, "entities": {"hashtags": [{"text": "Euromed", "indices": [72, 80]}, {"text": "IHEDN", "indices": [81, 87]}, {"text": "Marseille", "indices": [97, 107]}, {"text": "Paris", "indices": [119, 125]}], "symbols": [], "user_mentions": [{"screen_name": "sebastien_abis", "name": "S\u00e9bastien ABIS", "id": 622745686, "id_str": "622745686", "indices": [3, 18]}], "urls": []}, "source": "<a href=\"http://twitter.com/download/iphone\" rel=\"nofollow\">Twitter for iPhone</a>", "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 265897069, "id_str": "265897069", "name": "IRIS

Listing connections :

In [4]:
for friend in tweepy.Cursor(api.friends).items(1):
    print(json.dumps(friend._json)[:1000])

{"id": 265897069, "id_str": "265897069", "name": "IRIS", "screen_name": "InstitutIRIS", "location": "Paris", "description": "Acteur fran\u00e7ais de la recherche strat\u00e9gique et g\u00e9opolitique. Ses activit\u00e9s : la recherche, l\u2019organisation de manifestations, la publication, la formation.", "url": "http://t.co/EctAbFIhS6", "entities": {"url": {"urls": [{"url": "http://t.co/EctAbFIhS6", "expanded_url": "http://iris-france.org", "display_url": "iris-france.org", "indices": [0, 22]}]}, "description": {"urls": []}}, "protected": false, "followers_count": 24432, "friends_count": 422, "listed_count": 603, "created_at": "Mon Mar 14 09:29:08 +0000 2011", "favourites_count": 670, "utc_offset": 3600, "time_zone": "Paris", "geo_enabled": true, "verified": false, "statuses_count": 9310, "lang": "fr", "status": {"created_at": "Mon Dec 11 21:13:03 +0000 2017", "id": 940328598958936065, "id_str": "940328598958936065", "text": "RT @sebastien_abis: Tr\u00e8s heureux d'\u00eatre l'invit\u

Listing my own tweets :

In [5]:
for tweet in tweepy.Cursor(api.user_timeline).items():
    #print(json.dumps(tweet._json))
    print(tweet._json['text'])
    print(tweet._json['created_at'])

"« Cool, bières et bonbons gratuits ! » : plongée dans une start-up déjantée" #actualites #feedly https://t.co/cKWCAPZSVr
Sun Jul 24 17:30:30 +0000 2016


Some variables :
- `text`: the text of the tweet itself
- `created_at`: the date of creation
- `favorite_count`, `retweet_count`: the number of favourites and retweets
- `favorited`, `retweeted`: boolean stating whether the authenticated user (you) have favourited or retweeted this tweet
- `lang`: acronym for the language (e.g. “en” for english)
- `id`: the tweet identifier
- `place`, `coordinates`, `geo`: geo-location information if available
- `user`: the author’s full profile
- `entities`: list of entities like URLs, @-mentions, hashtags and symbols
- `in_reply_to_user_id`: user identifier if the tweet is a reply to a specific user
- `in_reply_to_status_id`: status identifier id the tweet is a reply to a specific status

Listing tweets from another user :

In [6]:
for tweet in tweepy.Cursor(api.user_timeline, id='BarackObama').items(5):
    #print(json.dumps(tweet._json))
    print(tweet._json['text'])
    print(tweet._json['created_at'])

Just got off a call to thank folks who are working hard to help more Americans across the country sign up for healt… https://t.co/A9QvCySWJ8
Mon Dec 11 20:16:29 +0000 2017
RT @ObamaFoundation: Watch: We hosted a Town Hall in New Delhi with @BarackObama and young leaders about how to drive change and make an im…
Mon Dec 04 22:57:47 +0000 2017
Michelle and I are delighted to congratulate Prince Harry and Meghan Markle on their engagement. We wish you a life… https://t.co/KC9nmjZPuX
Mon Nov 27 21:13:50 +0000 2017
From the Obama family to yours, we wish you a Happy Thanksgiving full of joy and gratitude. https://t.co/xAvSQwjQkz
Thu Nov 23 14:44:27 +0000 2017
ME:  Joe, about halfway through the speech, I’m gonna wish you a happy birth--
BIDEN:  IT’S MY BIRTHDAY!
ME:  Joe.… https://t.co/5qLUsDoaMi
Mon Nov 20 19:02:11 +0000 2017


## Text pre-processing

Text tokenization using NLTK:

In [7]:
import nltk
from nltk.tokenize import word_tokenize

for tweet in tweepy.Cursor(api.user_timeline, id='BarackObama').items(5):
    tweet_text = tweet._json['text']
    print(word_tokenize(tweet_text))

['Just', 'got', 'off', 'a', 'call', 'to', 'thank', 'folks', 'who', 'are', 'working', 'hard', 'to', 'help', 'more', 'Americans', 'across', 'the', 'country', 'sign', 'up', 'for', 'healt…', 'https', ':', '//t.co/A9QvCySWJ8']
['RT', '@', 'ObamaFoundation', ':', 'Watch', ':', 'We', 'hosted', 'a', 'Town', 'Hall', 'in', 'New', 'Delhi', 'with', '@', 'BarackObama', 'and', 'young', 'leaders', 'about', 'how', 'to', 'drive', 'change', 'and', 'make', 'an', 'im…']
['Michelle', 'and', 'I', 'are', 'delighted', 'to', 'congratulate', 'Prince', 'Harry', 'and', 'Meghan', 'Markle', 'on', 'their', 'engagement', '.', 'We', 'wish', 'you', 'a', 'life…', 'https', ':', '//t.co/KC9nmjZPuX']
['From', 'the', 'Obama', 'family', 'to', 'yours', ',', 'we', 'wish', 'you', 'a', 'Happy', 'Thanksgiving', 'full', 'of', 'joy', 'and', 'gratitude', '.', 'https', ':', '//t.co/xAvSQwjQkz']
['ME', ':', 'Joe', ',', 'about', 'halfway', 'through', 'the', 'speech', ',', 'I', '’', 'm', 'gon', 'na', 'wish', 'you', 'a', 'happy', 'birth'

Enhancing the tokenization by accounting for @-mentions, emoticons, URLs and hash-tags:

In [8]:
import re
 
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)
 
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens
 
for tweet in tweepy.Cursor(api.user_timeline, id='BarackObama').items(5):
    tweet_text = tweet._json['text']
    print(preprocess(tweet_text))
    

['Just', 'got', 'off', 'a', 'call', 'to', 'thank', 'folks', 'who', 'are', 'working', 'hard', 'to', 'help', 'more', 'Americans', 'across', 'the', 'country', 'sign', 'up', 'for', 'healt', '…', 'https://t.co/A9QvCySWJ8']
['RT', '@ObamaFoundation', ':', 'Watch', ':', 'We', 'hosted', 'a', 'Town', 'Hall', 'in', 'New', 'Delhi', 'with', '@BarackObama', 'and', 'young', 'leaders', 'about', 'how', 'to', 'drive', 'change', 'and', 'make', 'an', 'im', '…']
['Michelle', 'and', 'I', 'are', 'delighted', 'to', 'congratulate', 'Prince', 'Harry', 'and', 'Meghan', 'Markle', 'on', 'their', 'engagement', '.', 'We', 'wish', 'you', 'a', 'life', '…', 'https://t.co/KC9nmjZPuX']
['From', 'the', 'Obama', 'family', 'to', 'yours', ',', 'we', 'wish', 'you', 'a', 'Happy', 'Thanksgiving', 'full', 'of', 'joy', 'and', 'gratitude', '.', 'https://t.co/xAvSQwjQkz']
['ME', ':', 'Joe', ',', 'about', 'halfway', 'through', 'the', 'speech', ',', 'I', '’', 'm', 'gonna', 'wish', 'you', 'a', 'happy', 'birth', '-', '-', 'BIDEN', ':'

As stated in the blog post, the tokeniser is probably far from perfect, but it gives you the general idea. The tokenisation is based on regular expressions (regexp), which is a common choice for this type of problem. Some particular types of tokens (e.g. phone numbers or chemical names) will not be captured, and will be probably broken into several tokens. To overcome this problem, as well as to improve the richness of your pre-processing pipeline, you can improve the regular expressions, or even employ more sophisticated techniques like Named Entity Recognition.

## Analysing term frequencies

Let's count the terms used in the last 200 tweets of Barack Obama :

In [9]:
import operator 
import json
from collections import Counter

count_all = Counter()
for tweet in tweepy.Cursor(api.user_timeline, id='BarackObama').items(200):
    tweet_text = tweet._json['text']
    # Create a list with all the terms
    terms_all = [term for term in preprocess(tweet_text)]
    # Update the counter
    count_all.update(terms_all)
    
# Print the first 5 most frequent words
print(count_all.most_common(5))

[('.', 234), ('the', 139), ('to', 111), (':', 99), (',', 94)]


As you can see, the most frequent words are not exactly meaningful.  Let's remove the common words, called "stop-words".  We can use NLTK for this.  We also include tweet-specific stop-words such as RT (used for re-tweets), "via" and others:

In [21]:
import nltk
nltk.download('stopwords')
from nltk import bigrams 

[nltk_data] Downloading package stopwords to
[nltk_data]     d:\Profiles\cnozaradan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
from nltk.corpus import stopwords
import string
 
punctuation = list(string.punctuation)
stop = stopwords.words('english') + punctuation + ['rt', 'via', 'RT', '…', '’', '“', 'The']

We can now adapt the code.  Let's embed it in a method.  We also seperatly count hashtag terms and sequences of two terms (makes easier to identify the topics).

In [28]:
def tweet_stats(twitter_id, tweet_nbr=300, most_common=5, lang='english'):
    punctuation = list(string.punctuation)
    stop = stopwords.words(lang) + punctuation + ['rt','via','RT','…','’','“','The','é','les','a','La','Le','Il','être','→']
    count_all = Counter()
    count_single = Counter()
    count_hash = Counter()
    count_bigram = Counter()
    for tweet in tweepy.Cursor(api.user_timeline, id=twitter_id).items(tweet_nbr):
        tweet_text = tweet._json['text']
        # Create a list with all the terms
        terms_preprocessed = preprocess(tweet_text)
        terms_all = [term for term in terms_preprocessed if term not in stop]
        # Update the counter
        count_all.update(terms_all)
        # Count terms only once, equivalent to Document Frequency
        terms_single = set(terms_all)
        count_single.update(terms_single)
        # Count hashtags only
        terms_hash = [term for term in terms_preprocessed if term.startswith('#')]
        count_hash.update(terms_hash)
        # Considering sequences of two terms = bigrams
        terms_bigram = bigrams(terms_all)
        count_bigram.update(terms_bigram)
        
       
    # Print the first 5 most frequent words
    print('Most common terms:\n', count_all.most_common(most_common))
    print('\n Most common terms, counted once per tweet:\n', count_single.most_common(most_common))
    print('\n Most common hash-tag terms: ', count_hash.most_common(most_common))
    print('\n Most common bigrams: ', count_bigram.most_common(most_common))

In [29]:
tweet_stats('BarackObama')

Most common terms:
 [('—', 55), ('Senate', 46), ('leaders', 45), ('#DoYourJob', 39), ('President', 32)]

 Most common terms, counted once per tweet:
 [('—', 52), ('Senate', 46), ('leaders', 45), ('#DoYourJob', 39), ('President', 32)]

 Most common hash-tag terms:  [('#DoYourJob', 39), ('#ActOnClimate', 31), ('#Obamacare', 11), ('#GetCovered', 9), ('#ParisAgreement', 3)]

 Most common bigrams:  [(('Senate', 'leaders'), 43), (('President', 'Obama'), 29), (('Supreme', 'Court'), 23), (('Judge', 'Garland'), 18), (('Truth', 'Team'), 15)]


For comparison, let's do the same on the tweets from Donald Trump:

In [30]:
tweet_stats('realDonaldTrump')

Most common terms:
 [('I', 54), ('great', 35), ('amp', 23), ('President', 23), ('years', 20)]

 Most common terms, counted once per tweet:
 [('I', 47), ('great', 34), ('President', 23), ('amp', 21), ('years', 20)]

 Most common hash-tag terms:  [('#MAGA', 4), ('#MakeAmericaGreatAgain', 3), ('#GES2017', 2), ('#POTUSinAsia', 2), ('#PearlHarbor', 2)]

 Most common bigrams:  [(('U', 'S'), 11), (('Fake', 'News'), 9), (('Tax', 'Cuts'), 9), (('Crooked', 'Hillary'), 9), (('Stock', 'Market'), 7)]


This shows already some interesting trends...  Out of curiosity:

In [32]:
tweet_stats('EmmanuelMacron', lang='french')

Most common terms:
 [('Nous', 37), ('Je', 34), ('France', 30), ('Avec', 22), ('ée', 20)]

 Most common terms, counted once per tweet:
 [('Nous', 35), ('Je', 31), ('France', 28), ('Avec', 22), ('ée', 19)]

 Most common hash-tag terms:  [('#CongresAMF', 8), ('#NeRienLaisserPasser', 7), ('#OnePlanet', 5), ('#JohnnyHallyday', 5), ('#SocialSummit17', 5)]

 Most common bigrams:  [(('Je', 'veux'), 10), (('🥇', '🥇'), 9), (('apr', 'ès'), 8), (('aujourd', 'hui'), 7), (('lutte', 'contre'), 6)]


In [36]:
tweet_stats('NASA')

Most common terms:
 [('@Space_Station', 62), ('@AstroKomrade', 35), ('space', 31), ('black', 28), ('us', 26)]

 Most common terms, counted once per tweet:
 [('@Space_Station', 62), ('@AstroKomrade', 35), ('space', 30), ('black', 27), ('us', 26)]

 Most common hash-tag terms:  [('#BlackFriday', 16), ('#Cygnus', 9), ('#BlackHoleFriday', 8), ('#NASASiliconValley', 4), ('#ICYMI', 4)]

 Most common bigrams:  [(('black', 'holes'), 17), (('LIVE', 'NOW'), 12), (('pm', 'ET'), 10), (('black', 'hole'), 10), (('Join', 'us'), 9)]
